# Стажировка

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [4]:
# считать датасет, столбец с датой сразу преобразовать в datetime
rent = pd.read_csv('rent_apartments.csv', sep=',', parse_dates=[10], date_format='%d/%m/%Y')
rent[:4]

,object_type,type,rooms,level,kitchen_area,living_area,area,price_by_meter,price,material,...,build_walls,heating,gas,longitude,latitude,agencyName,areaRating,city,remoute_from_center,build_oldest
0,0,Не заполнено,2,3/5,10.0,40.0,60.0,200.0,12000.0,Панель,...,Не заполнено,0,0,47.863361,56.647657,Антон,3.634451,Йошкар-Ола,1.951714,new
1,0,Квартира,2,1/10,12.0,45.0,60.0,583.0,35000.0,Кирпич,...,0,0,0,47.877609,56.625316,Антон Александрович,3.634451,Йошкар-Ола,1.702297,new
2,0,Многоквартирный дом,3,1/5,10.0,54.0,82.0,268.0,22000.0,Кирпич,...,Кирпич,Центральное,Да,47.900983,56.644836,"Риелторская компания ""Партнеры""",3.634451,Йошкар-Ола,0.920945,old
3,0,Многоквартирный дом,1,2/5,8.0,18.0,35.0,314.0,11000.0,Кирпич,...,Кирпич,Центральное,Да,47.878426,56.647054,"Риелторская компания ""Партнеры""",3.634451,Йошкар-Ола,1.193907,old


In [ ]:
rent.info()

In [ ]:
rent.shape

## Обработка дубликатов

In [ ]:
# вывести количество полных дубликатов
rent.duplicated().sum()

In [ ]:
rent[rent.duplicated()][:5]

In [ ]:
# сохранить дубликаты в файл
rent[rent.duplicated()].to_csv('rent_apartments_dup.csv')

In [6]:
# удалить полностью дублирующие записи
rent = rent.drop_duplicates()

## Отбор столбцов для анализа

In [8]:
columns = ['rooms', 'level', 'area', 'price', 'material', 'published', 'city', 'remoute_from_center', 'build_oldest']
rent = rent[columns]
rent[:2]

,rooms,level,area,price,material,published,city,remoute_from_center,build_oldest
0,2,3/5,60.0,12000.0,Панель,2021-05-20,Йошкар-Ола,1.951714,new
1,2,1/10,60.0,35000.0,Кирпич,2021-09-30,Йошкар-Ола,1.702297,new


## Обработка пропусков

In [85]:
# сводные данные по наличию NaN
rent.isna().sum()

rooms                  0.0
level                  0.0
area                   0.0
price                  0.0
material               0.0
published              0.0
city                   0.0
remoute_from_center    0.0
build_oldest           0.0
dtype: float64

In [89]:
rent.loc[rent.material.isna()][:3]

,rooms,level,area,price,material,published,city,remoute_from_center,build_oldest


In [18]:
# удалить строки со значениями NaN
rent = rent.dropna()

In [16]:
rent.loc[(rent.rooms == 0) & (rent.city == 'Москва')].shape[0]

9012

In [191]:
# удалить записи с количеством комнат = 0
rent = rent.loc[rent.rooms != 0]

In [193]:
rent.shape

(11399, 9)

## Распаковка столбца `level`

In [20]:
def get_level(s, i):
    """
    Возвращает элемент списка, приведенный к типу int, по заданному индексу.
    При возникновении исключения ValueError возвращает NaN
    Parameters
    ----------
    s : {list}
        Список из которого требуется возвратить элемент
    i : {int}
        Индекс возвращаемого элемента
    Returns
    -------
    Элемент списка, преобразованный в int или NaN    
    """
    try:
        return int(float(s[i]))
    except ValueError:
        return np.nan

In [22]:
rent['level_lst'] = rent['level'].str.split('/') # создать временный столбец, разделив значения в cтолбце 'level' по '/'
rent['level'] = rent['level_lst'].apply(get_level, i=0) # в столбец 'level' записать этаж
rent['level_all'] = rent['level_lst'].apply(get_level, i=1) # в столбец 'level_all' записать общее количество этажей дома
rent = rent.drop(labels=['level_lst'], axis=1)  # удалить временный столбец
rent.isna().sum() # проверить корректность распаковки столбца 'level' (отсутствие значений NaN)

rooms                  0
level                  0
area                   0
price                  0
material               0
published              0
city                   0
remoute_from_center    0
build_oldest           0
level_all              0
dtype: int64

## Замена категорий

In [24]:
mapping_template = {'new': 'новостройка', 'old': 'старый фонд', 'middle': 'вторичка'}
rent['build_oldest'] = rent['build_oldest'].map(mapping_template)

## Расчет столбца с ценой за кв.м

In [26]:
rent['price_per_meter'] = rent['price'].div(rent['area']).round(0)

## Округление данных

In [28]:
rent['remoute_from_center'] = rent['remoute_from_center'].round(0)

## Проверка валидности данных

In [30]:
# проверка, что этаж не превышает этажности здания
rent.query('level > level_all')

,rooms,level,area,price,material,published,city,remoute_from_center,build_oldest,level_all,price_per_meter


## Фильтрация данных

In [32]:
start_date = dt.datetime(2021, 1, 1)
end_date = dt.datetime(2022, 3, 1)
rent = rent.loc[(rent['published'] >= start_date) & (rent['published'] < end_date) & (rent['city'] == 'Москва')]
rent.shape

(7898, 11)

In [34]:
rent.loc[rent.rooms == 0].shape[0]

471

## !!!Удаление выбросов

In [210]:
rent.loc[rent.price_per_meter > 15000]

,rooms,level,area,price,material,published,city,remoute_from_center,build_oldest,level_all,price_per_meter
759,2,9,52.0,45000000.0,Панель,2022-02-10,Москва,14.0,старый фонд,14,865385.0
1500,1,4,30.0,29000000.0,Кирпич,2022-02-10,Москва,11.0,старый фонд,5,966667.0


## Переименование столбцов

In [179]:
columns={'rooms': 'Количество_комнат',
         'level': 'Этаж',
         'area': 'Площадь',
         'price': 'Арендная_плата',
         'material': 'Конструктив_дома',
         'published': 'Дата_публикации',
         'city': 'Город',
         'remoute_from_center': 'Расстояние от центра',
         'build_oldest': 'Тип недвижимости',
         'level_all': 'Этажность',
         'price_per_meter': 'Арендная_плата_за_метр'}
rent = rent.rename(columns=columns)
rent[:3]

,Количество_комнат,Этаж,Площадь,Арендная_плата,Конструктив_дома,Дата_публикации,Город,Расстояние от центра,Тип недвижимости,Этажность
189,1,2,32.0,29000.0,Панель,2021-11-22,Москва,11.075363,старый фонд,5
190,1,1,31.0,48000.0,Панель,2022-01-11,Москва,17.033073,старый фонд,9
191,1,5,38.9,44000.0,Панель,2022-02-03,Москва,10.463894,вторичка,9
